In [134]:
# !pip install cmfrec
# !pip install lightfm
# !pip install matrix-factorization

In [136]:
import pandas as pd
import numpy as np

In [137]:
from numpy.linalg import svd

In [ ]:
ruta_archivo = 'u.data'

In [139]:
df = pd.read_csv(ruta_archivo, sep='\t', names=['user_id','movie_id','rating','timestamp'])
df

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


1️⃣ SVD tradicional
- Definición: $A = U \Sigma V^T$
- Valores singulares ($\sigma_i$): importancia de cada componente / varianza explicada
- Uso: PCA, compresión, análisis espectral
- Requisito: matriz completa


In [140]:
# from numpy.linalg import svd

In [141]:
ratings_df = pd.pivot_table(data=df, values='rating', index='user_id', columns='movie_id')

In [142]:
# Supongamos que tienes una matriz completa de ratings (usuarios × películas)
# ratings_df: DataFrame donde filas=usuarios, columnas=películas, valores=ratings
A = ratings_df.values  # forma (m, n), sin valores faltantes

In [143]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)
Sigma = np.diag(s)

LinAlgError: SVD did not converge

problema de valores nulos.. Imputaremos por '0' para simplificar

In [144]:
# Reemplazar NaN por 0 (u otra estrategia, como la media del usuario)
A = np.nan_to_num(A, nan=0.0)  # reemplaza NaN por 0

In [145]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)
Sigma = np.diag(s)

In [146]:
# Reconstrucción exacta
A_hat = U @ Sigma @ VT
A_hat

array([[ 5.00000000e+00,  3.00000000e+00,  4.00000000e+00, ...,
        -2.55048397e-16, -1.56125113e-17,  5.68772460e-16],
       [ 4.00000000e+00,  3.43385043e-13, -8.13932255e-15, ...,
        -1.50920942e-16,  4.70110062e-16,  1.73472348e-17],
       [-2.69020917e-14,  1.23581700e-14,  1.43982049e-16, ...,
        -2.41993925e-16, -1.38777878e-16, -1.04733930e-16],
       ...,
       [ 5.00000000e+00,  9.57567359e-16,  5.83907922e-15, ...,
        -2.48932819e-16, -4.85722573e-16, -1.78676518e-16],
       [-4.15466272e-15,  8.39779635e-15, -1.70002901e-15, ...,
        -4.91794105e-16, -4.33680869e-17,  1.46150453e-16],
       [ 1.72188652e-14,  5.00000000e+00, -2.18575158e-15, ...,
        -3.50414142e-16,  2.87964097e-16, -2.65412692e-16]])

Al utilizar todos los vectores singulares de U y V_T y todos los valores sigulates de la matriz diagonal, la reconstrucción es exacta a la matriz original, no es una aproximación.

In [147]:
# Interpretación: los valores s[i] indican la “energía” de cada componente
s[:10]

array([640.63362257, 244.83634567, 217.84622472, 159.15359872,
       158.21191449, 145.87261327, 126.57977314, 121.90769976,
       106.8291837 ,  99.74793974])

2️⃣ SVD truncada para recomendación
- Definición: $A \approx U_k \Sigma_k V_k^T$
- Valores singulares: fuerza de factores latentes (patrones usuario–ítem)
- Uso: predicción de ratings, embeddings de usuarios e ítems
- Requisito: matriz completa o con estrategias de imputación para valores faltantes
- Ventaja: reduce dimensionalidad, captura patrones latentes
- Nota: no maneja sparsidad de manera nativa; requiere completitud o aproximación



In [148]:
ratings_df = pd.pivot_table(data=df, values='rating', index='user_id', columns='movie_id')

In [149]:
# Supongamos que tienes una matriz completa de ratings (usuarios × películas)
# ratings_df: DataFrame donde filas=usuarios, columnas=películas, valores=ratings
A = ratings_df.values  # forma (m, n), sin valores faltantes

In [150]:
# SVD completa
U, s, VT = svd(A, full_matrices=False)
Sigma = np.diag(s)

LinAlgError: SVD did not converge

problema de valores nulos.. Imputaremos por '0' para simplificar

In [151]:
# Reemplazar NaN por 0 (u otra estrategia, como la media del usuario)
A = np.nan_to_num(A, nan=0.0)  # reemplaza NaN por 0

In [152]:
U, s, VT = svd(A, full_matrices=False)

In [153]:
s

array([640.63362257, 244.83634567, 217.84622472, 159.15359872,
       158.21191449, 145.87261327, 126.57977314, 121.90769976,
       106.8291837 ,  99.74793974,  93.79885965,  93.25844284,
        89.91150168,  84.34178722,  83.81220836,  81.81204105,
        79.07796788,  77.88652669,  76.387996  ,  75.3415951 ,
        73.68235502,  72.80837191,  72.51350545,  71.52749477,
        69.77179735,  69.10881715,  68.8735702 ,  67.94277928,
        67.40829434,  67.06352378,  66.85757418,  65.59270059,
        65.27526042,  64.79965625,  64.44727664,  64.09819141,
        63.91638042,  63.08261122,  62.67586971,  62.23742793,
        62.03574728,  61.77291401,  61.33544177,  61.0632462 ,
        60.56817026,  60.30813928,  59.77166759,  59.51420996,
        59.40675   ,  59.10683763,  58.83667955,  58.53445585,
        58.33802154,  58.1323194 ,  57.41759146,  57.36384311,
        57.30977341,  56.99448748,  56.72636608,  56.239748  ,
        56.17894513,  55.8734678 ,  55.65459359,  55.52

In [154]:
k = 20  # número de componentes latentes

In [155]:
s[:k]

array([640.63362257, 244.83634567, 217.84622472, 159.15359872,
       158.21191449, 145.87261327, 126.57977314, 121.90769976,
       106.8291837 ,  99.74793974,  93.79885965,  93.25844284,
        89.91150168,  84.34178722,  83.81220836,  81.81204105,
        79.07796788,  77.88652669,  76.387996  ,  75.3415951 ])

In [156]:
U_k = U[:, :k]
Sigma_k = np.diag(s[:k])
V_k = VT.T[:, :k]

In [157]:
# Reconstrucción aproximada
A_hat = U_k @ Sigma_k @ V_k.T
A_hat

array([[ 4.01688618e+00,  2.10514989e+00,  1.37439578e+00, ...,
        -3.83174025e-03,  2.64379479e-02,  7.33806834e-02],
       [ 1.99295186e+00, -1.81693234e-02, -4.30251308e-03, ...,
         9.28078660e-03, -2.66167393e-03, -2.95256660e-02],
       [-1.74774668e-01, -5.60278496e-02,  1.81311614e-01, ...,
         2.16443047e-02,  1.91721169e-03,  2.49374917e-04],
       ...,
       [ 2.23624102e+00,  3.89103327e-02,  2.76077652e-01, ...,
        -5.53764648e-03,  1.07306047e-02, -4.23773725e-03],
       [ 1.31024409e+00,  1.55156267e-01, -4.75132794e-01, ...,
         1.50293142e-02,  1.15206267e-02, -2.94522888e-02],
       [ 1.72204325e+00,  2.05335046e+00,  1.18288029e+00, ...,
        -9.89971558e-03,  1.73695571e-02,  2.05937102e-02]])

Cuando seleccionamos los 'k' valores y vectores singulares, solo podremos aproximar la solción pero con mucho menor costo computacional asociado.

In [158]:
# Crear embeddings de usuarios e ítems
X_users = U_k @ np.sqrt(Sigma_k)
Y_items = V_k @ np.sqrt(Sigma_k)

3️⃣ SVD iterativa (FunkSVD, SVD++)
- Definición: optimización iterativa
$\min_{P,Q} \sum_{(u,i) \in \Omega} (r_{ui} - p_u^T q_i)^2 + \lambda(\|p_u\|^2 + \|q_i\|^2)$
- Valores singulares: implícitos; $P$ y $Q$ se ajustan por gradiente o ALS
- Uso: recomendadores modernos, escalables, con regularización e interacciones implícitas
- Requisito: no necesita matriz completa; solo requiere los ratings observados
- Ventaja: funciona directamente con matrices muy dispersas y generaliza patrones latentes

Implementación paso a paso:


In [159]:
# Mapear IDs a índices consecutivos
user_mapping = {id_: idx for idx, id_ in enumerate(df['user_id'].unique())}
item_mapping = {id_: idx for idx, id_ in enumerate(df['movie_id'].unique())}
df['user_idx'] = df['user_id'].map(user_mapping)
df['item_idx'] = df['movie_id'].map(item_mapping)

In [160]:
n_users = df['user_idx'].nunique()
n_items = df['item_idx'].nunique()
k = 20

In [161]:
# Inicialización de factores y biases
P = np.random.normal(scale=0.1, size=(n_users, k))
Q = np.random.normal(scale=0.1, size=(n_items, k))
bu = np.zeros(n_users)  # bias usuario
bi = np.zeros(n_items)  # bias ítem
global_mean = df['rating'].mean()

In [162]:
lr = 0.005
lambda_reg = 0.02
n_epochs = 10

In [163]:
rows = df['user_idx'].values
cols = df['item_idx'].values
vals = df['rating'].values

In [164]:
# Entrenamiento FunkSVD con SGD
for epoch in range(n_epochs):
    for u, i, r_ui in zip(rows, cols, vals):
        pred = global_mean + bu[u] + bi[i] + P[u] @ Q[i].T
        err = r_ui - pred
        # Actualizar factores y biases
        bu[u] += lr * (err - lambda_reg * bu[u])
        bi[i] += lr * (err - lambda_reg * bi[i])
        P[u] += lr * (err * Q[i] - lambda_reg * P[u])
        Q[i] += lr * (err * P[u] - lambda_reg * Q[i])
    print(f"Epoch {epoch+1} finalizada")

Epoch 1 finalizada
Epoch 2 finalizada
Epoch 3 finalizada
Epoch 4 finalizada
Epoch 5 finalizada
Epoch 6 finalizada
Epoch 7 finalizada
Epoch 8 finalizada
Epoch 9 finalizada
Epoch 10 finalizada


In [165]:
ratings_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
df.head()

,user_id,movie_id,rating,timestamp,user_idx,item_idx
0,196,242,3,881250949,0,0
1,186,302,3,891717742,1,1
2,22,377,1,878887116,2,2
3,244,51,2,880606923,3,3
4,166,346,1,886397596,4,4


In [169]:
print("Usuarios en mapping:", list(user_mapping.keys())[:10])
print("Películas en mapping:", list(item_mapping.keys())[:10])

Usuarios en mapping: [np.int64(196), np.int64(186), np.int64(22), np.int64(244), np.int64(166), np.int64(298), np.int64(115), np.int64(253), np.int64(305), np.int64(6)]
Películas en mapping: [np.int64(242), np.int64(302), np.int64(377), np.int64(51), np.int64(346), np.int64(474), np.int64(265), np.int64(465), np.int64(451), np.int64(86)]


In [167]:
# Predicción ejemplo
user_id = df['user_id'].iloc[4]
item_id = df['movie_id'].iloc[3]
u_idx = user_mapping[user_id]
i_idx = item_mapping[item_id]

In [168]:
rating_pred = global_mean + bu[u_idx] + bi[i_idx] + P[u_idx] @ Q[i_idx].T
rating_pred

np.float64(3.621485302232855)